In [1]:
# this is just a temp fix to get things working locally. use the server config when starting jupyter
import sys, os

# NOTE: change this path if you need to for your system
client_side_path = os.getcwd()
print("client_side_path: %s" % (client_side_path))
sys.path.append(client_side_path)

client_side_path: /media/dpihera/Extra Drive 1/development/jupyternb_c_app_websockets/client_side


In [2]:
from wsclient.client import TestWebSocketClient
from tornado import ioloop

print("IOLoop: %s" % (ioloop.IOLoop.current()))
client = TestWebSocketClient()
client.connect('ws://localhost:8675/ws')

IOLoop: <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fcc9e404a58>
WebSocketClient.__init__: cto, rto = 10, 60
WebSocketClient.connect: attempting connection to: ws://localhost:8675/ws
WebSocketClient.connect: ws_conn is: <tornado.concurrent.Future object at 0x7fcca76d9a90>
WebSocketClient.connect: adding to ioloop (<zmq.eventloop.ioloop.ZMQIOLoop object at 0x7fcc9e404a58>)
WebSocketClient.connect: done


Connected to Websocket Server!
client_side.TestWebSocketClient._on_message: precessing Hello World from the server_side


In [6]:
from ipywidgets import Button
from IPython.display import display

button = Button(description='Send a test Message!')
display(button)

def send_test_msg(b):
    d = {'does_it': 'blend'}
    print('send_test_msg: sending test %s' % (d))
    client.send(d)

button.on_click(send_test_msg)

send_test_msg: sending test {'does_it': 'blend'}
client_side.TestWebSocketClient._on_message: precessing {does_it, blend} from the server_side
